In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Open Data Dashboard

QA script for dashboard_v3.py

In [2]:
#!/usr/bin/env python3
# coding: utf-8

from datetime import date

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import credentials

In [3]:
# delete in production code
# from importlib import reload
# reload(credentials)

In [4]:
#### Step 1. Load Published Data Asset Inventory

# Local Law 251 of 2017: Published Data Asset Inventory
# https://data.cityofnewyork.us/City-Government/Local-Law-251-of-2017-Published-Data-Asset-Invento/5tqd-u88y
public_df = credentials.call_socrata_api('5tqd-u88y')
public_df.shape

(2999, 22)

In [5]:
public_cols = [
    'datasetinformation_agency',
    'name',
    'uid',
    'url',
    'update_datemadepublic',
    'update_automation',
    'update_updatefrequency',
    'last_data_updated_date',
    'type',
    'row_count'
]
public_df = public_df[public_cols]
public_df.shape

(2999, 10)

In [6]:
#### Step 2. Get dates of the data updates

# get the dates each of datasets has been updated
dates_df = public_df[public_df.uid.isin(['5tqd-u88y','qj2z-ibhs'])]\
                [['uid', 'last_data_updated_date']]
dates_df['last_data_updated_date'] = pd.to_datetime(dates_df.last_data_updated_date, 
                                                     errors='coerce')\
                                            .dt.strftime("%Y-%m-%d")

today_df = pd.DataFrame({'uid':['NA'],
                         'last_data_updated_date':[date.today().strftime("%Y-%m-%d")],
                         'Source':['1. Dashboard']})
dates_df.loc[dates_df.uid=='5tqd-u88y','Source'] = '2. Published Asset Inventory'
dates_df.loc[dates_df.uid=='qj2z-ibhs','Source'] = '3. Open Plan Tracker'
dates_df = dates_df.append(today_df)
dates_df.reset_index(inplace=True, drop=True)
dates_df = dates_df[['Source', 'last_data_updated_date']]
dates_df.rename(columns={'last_data_updated_date':'updated_on'},inplace=True)
dates_df

,Source,updated_on
0,2. Published Asset Inventory,2020-11-24
1,3. Open Plan Tracker,2020-11-20
2,1. Dashboard,2020-11-25


In [7]:
#### Step 3. Load Private Data Asset Inventory

# Asset Inventory (Private Access)
# https://data.cityofnewyork.us/dataset/Asset-Inventory/kvci-ugf9
private_df = credentials.call_socrata_api('kvci-ugf9')
private_df = private_df[['uid','audience','derived_view','parent_uid']]
private_df.shape

(23790, 4)

In [8]:
public_df = public_df.merge(private_df,on='uid',how='left')
public_df.drop(columns=['audience'],inplace=True)
public_df.shape

(2999, 12)

In [9]:
public_df.groupby(['type'])['row_count'].sum().round().sort_index().apply(lambda x: '%.f' % x)

type
datalens             0
dataset     3238160483
file                 0
filter       114540607
href                 0
map            8988855
Name: row_count, dtype: object

In [10]:
# Create merged_filter, the dataframe that has only assets defined as datasets
# ZF approved the list
dataset_filter_list = ['dataset','filter','map']
public_filtered_df = public_df[public_df.type.isin(dataset_filter_list)]
public_filtered_df.shape

(2596, 12)

In [11]:
## remove derived assets if parent asset is public

# get parent ids for derived assets
parent_uids = public_filtered_df[public_filtered_df['derived_view']==True]['parent_uid']
# get ids for the assets derived from public assets
exc_parent_uids = public_filtered_df[public_filtered_df['uid'].isin(parent_uids)]['uid']
# remove derived assets if parent asset is public
public_filtered_df = public_filtered_df[~public_filtered_df['parent_uid'].isin(exc_parent_uids)]
public_filtered_df.shape

(2520, 12)

In [12]:
#### Step 4. Create one main dataset-level dataframe

# fix one date typo
public_filtered_df.loc[public_filtered_df['update_datemadepublic']=='August 9, 2-019',\
                       'update_datemadepublic'] = 'August 9, 2019'

# convert to date
public_filtered_df['update_datemadepublic'] = pd.to_datetime(
                                            pd.to_datetime(public_filtered_df['update_datemadepublic'],
                                                           errors='coerce')\
                                            .dt.strftime('%m/%d/%Y'), format=('%m/%d/%Y'))
public_filtered_df['last_data_updated_date'] = pd.to_datetime(
                                                 pd.to_datetime(public_filtered_df['last_data_updated_date'])\
                                                 .dt.strftime('%m/%d/%Y'))

# if agency is missing, create NA category
public_filtered_df['datasetinformation_agency'] = public_filtered_df['datasetinformation_agency'].fillna('Not filled out')
public_filtered_df.loc[public_filtered_df['datasetinformation_agency']=='','datasetinformation_agency'] = 'Not filled out'

keep_quant_cols=[
 'uid',
 'datasetinformation_agency',
 'name',
 'url',
 'type',
 'update_datemadepublic',
 'last_data_updated_date',
 'row_count'
]

quantity_dataset_df = public_filtered_df[keep_quant_cols]
quantity_dataset_df.shape

(2520, 8)

In [13]:
#### Step 5. Create one main agency-level dataframe

quantity_agency_df = quantity_dataset_df.groupby(['datasetinformation_agency'])\
                            .agg({'uid':'size','row_count':'sum'})\
                            .reset_index()\
                            .rename(columns={'uid':'numdatasets'})
quantity_agency_df.shape

(89, 3)

In [14]:
#### QUALITY (Data Freshness) ####

#### Step 1. Build baseline dataset

freshness_df = public_filtered_df[[
    'datasetinformation_agency',
    'name',
    'uid',
    'update_updatefrequency',
    'url',
    'update_datemadepublic',
    'last_data_updated_date',
    'update_automation']]

# Remove datasets with update frequencies for which we cannot determine freshness
freshness_df = freshness_df[(~freshness_df['update_updatefrequency']\
                            .isin(['Historical Data', 'As needed'])) &\
                             ~freshness_df['update_updatefrequency'].isna()]\
                            .reset_index(drop=True)

def assign_dataframe_statuses(data):

    """
    Determines if the data has been updated on time
    """
    
    df = data.copy()

    # some values have spaces
    df['update_updatefrequency'] = df['update_updatefrequency'].str.strip()
    
    # assign time by update frequency
    status_conditions = [
        (df['update_updatefrequency']=='Annually'),
        (df['update_updatefrequency']=='Monthly'),
        (df['update_updatefrequency']=='Quarterly'),
        (df['update_updatefrequency']=='Daily'),
        (df['update_updatefrequency']=='Biannually'),
        (df['update_updatefrequency']=='Weekly'),
        (df['update_updatefrequency']=='Triannually'),
        (df['update_updatefrequency']=='Weekdays'),
        (df['update_updatefrequency']=='2 to 4 times per year'),
        (df['update_updatefrequency']=='Biweekly'),
        (df['update_updatefrequency']=='Several times per day'),
        (df['update_updatefrequency']=='Hourly'),
        (df['update_updatefrequency']=='Every four years')
    ]
    status_choices = [
        pd.Timedelta('365 days'),
        pd.Timedelta('31 days'),
        pd.Timedelta('92 days'),
        pd.Timedelta('25 hours'),
        pd.Timedelta('182 days'),
        pd.Timedelta('7 days'),
        pd.Timedelta('122 days'),
        pd.Timedelta('5 days'),
        pd.Timedelta('182 days'),
        pd.Timedelta('4 days'),
        pd.Timedelta('25 hours'),
        pd.Timedelta('25 hours'),
        pd.Timedelta('1460 days')
        ]
    
    df['update_threshold'] = np.select(status_conditions, status_choices, default=pd.Timedelta('50000 days'))
    
    # calculate when asset should have been last updated
    df['last_updated_ago'] = pd.to_datetime(date.today()) - df['last_data_updated_date']
    
    # assign status to automated, dictionary and geocoded columns
    df['fresh'] = np.where((df['last_updated_ago']>=df['update_threshold']),'No','Yes')
    
    df.drop(columns=['update_threshold'],inplace=True)
    
    return df

freshness_df = assign_dataframe_statuses(freshness_df)

keep_fresh_cols = [
 'uid',
 'datasetinformation_agency',
 'name',
 'url',
 'update_automation',
 'update_updatefrequency',
 'last_data_updated_date',
 'fresh'    
]

freshness_dataset_df = freshness_df[keep_fresh_cols]
freshness_dataset_df.shape

(1522, 8)

In [15]:
update_values_used = {
    'Annually',
    'Monthly',
    'Quarterly',
    'Daily',
    'Biannually',
    'Weekly',
    'Triannually',
    'Weekdays',
    '2 to 4 times per year',
    'Biweekly',
    'Several times per day',
    'Hourly',
    'Every four years'
}

update_values_available = set(freshness_dataset_df['update_updatefrequency'].unique())
# identify new update frequency values
update_values_available.difference(update_values_used)

set()

In [16]:
#### Step 2. Calculate average data freshness by agency

# get the count of fresh dataset by agency
fresh_count_df = freshness_df[freshness_df['fresh']=='Yes'].groupby(['datasetinformation_agency'])\
                                .size()\
                                .reset_index()\
                                .rename(columns={0:'fresh_count'})

# get the total count of datasets by agency (excluding historical and as needed)
freshness_agency_df = freshness_df.groupby(['datasetinformation_agency'])\
                                .size()\
                                .reset_index()\
                                .rename(columns={0:'total_auto_count'})\
                                .merge(fresh_count_df, on='datasetinformation_agency',how='left')

# calculate percent freshly updated
freshness_agency_df['fresh_pct'] = freshness_agency_df['fresh_count'].fillna(0) / freshness_agency_df['total_auto_count']
freshness_agency_df.shape

(72, 4)

In [17]:
freshness_agency_df.head()

,datasetinformation_agency,total_auto_count,fresh_count,fresh_pct
0,311,5,4.0,0.800000
1,Board of Standards and Appeals (BSA),1,1.0,1.000000
2,Brooklyn Borough President (BPBK),15,NaN,0.000000
3,Business Integrity Commission (BIC),9,8.0,0.888889
4,City Clerk & Clerk of the Council (OCC),1,NaN,0.000000


In [18]:
freshness_df[freshness_df['datasetinformation_agency']=='']

,datasetinformation_agency,name,uid,update_updatefrequency,url,update_datemadepublic,last_data_updated_date,update_automation,last_updated_ago,fresh


In [19]:
#### COMPLIANCE ####

#### Step 1. Build baseline dataset

# NYC Open Data Release Tracker
# https://data.cityofnewyork.us/City-Government/NYC-Open-Data-Release-Tracker/qj2z-ibhs
tracker_df = credentials.call_socrata_api('qj2z-ibhs')

# exclude Removed from the plan and Removed from the portal, 
release_status_filter = [
    'Released',
    'Scheduled for release',
    'Under Review'
]
tracker_df = tracker_df[tracker_df['release_status'].isin(release_status_filter)]

# apply grace period for release date
grace_period_days = 14
today = date.today()

# convert dates to dates
tracker_df['original_plan_date'] = pd.to_datetime(tracker_df['original_plan_date'])
tracker_df['latest_plan_date'] = pd.to_datetime(tracker_df['latest_plan_date'])
tracker_df['release_date'] = pd.to_datetime(tracker_df['release_date'])

# number of days between release and planned date
tracker_df['plan_to_release'] = (tracker_df['release_date'] - tracker_df['latest_plan_date']).dt.days

# create a check if released on time
tracker_df['within_grace_period'] = np.where((tracker_df['plan_to_release'] < grace_period_days), 'Yes', 'No')
tracker_df['within_grace_period_num'] = tracker_df['plan_to_release'] < grace_period_days

# subset datasets that were supposed to be released in the last 12 months
tracker_df['last_12_months'] = ((pd.to_datetime(today) - tracker_df['latest_plan_date']).dt.days < 365) & \
                                (tracker_df['latest_plan_date'] <= pd.to_datetime(today))

tracker_df['url'] = tracker_df['url1'].apply(lambda x: list(x.values())[0] \
                                                   if type(x) is dict else 'NA')
tracker_df.drop(columns=['url1'],inplace=True)
# drop duplicates for released datasets
# keep the one with the oldest release date
tracker_df = tracker_df[~tracker_df.u_id.isna()]\
                                .sort_values(by='release_date')\
                                .drop_duplicates(subset=['u_id'], keep='first')\
                                .append(tracker_df[tracker_df['u_id'].isna()])

tracker_12mo_df = tracker_df[tracker_df['last_12_months']]

tracker_12mo_df['latest_plan_date'] = tracker_12mo_df['latest_plan_date'].dt.strftime("%Y-%m-%d")
tracker_12mo_df['release_date'] = tracker_12mo_df['release_date'].dt.strftime("%Y-%m-%d")
tracker_12mo_df.shape

(113, 16)

In [20]:
tracker_12mo_df.head()

,agency,dataset,dataset_description,update_frequency,original_plan_date,latest_plan_date,release_status,agency_notes,from_the_2020_open_data_plan_,release_date,u_id,plan_to_release,within_grace_period,within_grace_period_num,last_12_months,url
718,Department of Transportation (DOT),Street Improvement Project (SIP),All completed and planned street changes that ...,To Be Determined,2019-12-31,2019-12-31,Released,Released as series of 2 datasets. The other UI...,NaN,2018-05-31,wqhs-q6wd,-579.0,Yes,True,True,https://data.cityofnewyork.us/Transportation/V...
1386,Department of Transportation (DOT),Highway Signs Installations,Database that tracks all signs installed in-ho...,Biannually,2019-12-31,2019-12-31,Released,NaN,NaN,2019-01-04,jkdk-6p97,-361.0,Yes,True,True,https://data.cityofnewyork.us/Transportation/H...
961,Department of Transportation (DOT),Staten Island Ferry Trips,Staten Island Ferry trips that are on time (%),To Be Determined,2015-07-01,2019-12-31,Released,NaN,NaN,2019-02-26,7gic-pibm,-308.0,Yes,True,True,https://data.cityofnewyork.us/Transportation/S...
540,Department of Transportation (DOT),Street Name Signs (SNS) Management System,This application assists the Street Name Signs...,To Be Determined,2018-12-31,2019-12-31,Released,"Due to the complexity of the dataset, data wil...",NaN,2019-02-27,guiy-frxt,-307.0,Yes,True,True,https://data.cityofnewyork.us/Transportation/S...
944,Department of Transportation (DOT),Midtown Bicycle Counts,"DOT conducts regular bike counts, which are su...",Annually,2018-12-31,2019-12-31,Released,NaN,NaN,2019-05-02,yusc-bz4f,-243.0,Yes,True,True,https://data.cityofnewyork.us/Transportation/M...


In [21]:
#### Step 2. Build dataset-level dataset

keep_tracker_cols = [
 'u_id',
 'agency',
 'dataset',
 'dataset_description',
 'latest_plan_date',
 'release_status',
 'release_date',
 'within_grace_period',
 'within_grace_period_num',
 'url'
]

tracker_12mo_dataset_df = tracker_12mo_df[keep_tracker_cols]

# append type and agency from public inventory
tracker_12mo_dataset_df = tracker_12mo_dataset_df.merge(public_df[['uid','type','datasetinformation_agency']], 
                                                                    left_on='u_id',
                                                                    right_on='uid',
                                                                    how='left')

# update agency name to match public inventory (can only be done for already published datasets)
tracker_12mo_dataset_df['datasetinformation_agency'] = np.where((tracker_12mo_dataset_df.release_status=='Released') & \
                                                                ~tracker_12mo_dataset_df['datasetinformation_agency'].isna(),
                                                                tracker_12mo_dataset_df['datasetinformation_agency'],
                                                                tracker_12mo_dataset_df['agency'])

# exclude assets that are not datasets, filters and gis maps
# keeps assets scheduled for release with type NA

tracker_12mo_dataset_df = tracker_12mo_dataset_df[tracker_12mo_dataset_df['u_id'].isin(quantity_dataset_df['uid']) | \
                                                  (tracker_12mo_dataset_df['release_status']=='Scheduled for release')]
tracker_12mo_dataset_df.drop(columns=['u_id','agency'],inplace=True)
tracker_12mo_dataset_df.shape

(103, 11)

In [22]:
tracker_12mo_dataset_df.head()

,dataset,dataset_description,latest_plan_date,release_status,release_date,within_grace_period,within_grace_period_num,url,uid,type,datasetinformation_agency
0,Street Improvement Project (SIP),All completed and planned street changes that ...,2019-12-31,Released,2018-05-31,Yes,True,https://data.cityofnewyork.us/Transportation/V...,wqhs-q6wd,map,Department of Transportation (DOT)
1,Highway Signs Installations,Database that tracks all signs installed in-ho...,2019-12-31,Released,2019-01-04,Yes,True,https://data.cityofnewyork.us/Transportation/H...,jkdk-6p97,dataset,Department of Transportation (DOT)
3,Street Name Signs (SNS) Management System,This application assists the Street Name Signs...,2019-12-31,Released,2019-02-27,Yes,True,https://data.cityofnewyork.us/Transportation/S...,guiy-frxt,dataset,Department of Transportation (DOT)
5,Waterborne Disease Risk Assessment Report,Tables from Waterborne Disease Risk Assessment...,2019-11-30,Released,2019-05-14,Yes,True,https://data.cityofnewyork.us/Health/DOHMH-and...,hphp-39kk,dataset,Department of Health and Mental Hygiene (DOHMH)
6,WalkNYC Signs,This data set contains the locations of existi...,2019-12-31,Released,2019-06-05,Yes,True,https://data.cityofnewyork.us/Transportation/W...,q49j-2bun,map,Department of Transportation (DOT)


In [23]:
#### Step 3. Build agency-level dataset

# count number of overdue for release datasets
agency_overdue_df = tracker_12mo_dataset_df[tracker_12mo_dataset_df['release_status']=='Scheduled for release']\
                                .groupby(['datasetinformation_agency']).size().reset_index()\
                                .rename(columns={0:'overdue_datasets'})

tracker_12mo_agency_df = tracker_12mo_dataset_df.groupby(['datasetinformation_agency'])\
                                        .agg({'datasetinformation_agency':'size',
                                              'within_grace_period_num':'sum'})\
                                        .rename(columns={'datasetinformation_agency':'tracker_dataset_count',
                                                         'within_grace_period_num':'tracker_count_ontime'})\
                                        .merge(agency_overdue_df, on='datasetinformation_agency', how='left')\
                                        .reset_index(drop=True)\
                                        .fillna(0)

# calculate percent released on time
tracker_12mo_agency_df['pct_ontime'] = tracker_12mo_agency_df['tracker_count_ontime'].fillna(0)/tracker_12mo_agency_df['tracker_dataset_count']
tracker_12mo_agency_df.shape

(29, 5)

In [24]:
tracker_12mo_agency_df.head()

,datasetinformation_agency,tracker_dataset_count,tracker_count_ontime,overdue_datasets,pct_ontime
0,Brooklyn Borough President (BPBK),11,11,0.0,1.0
1,Department for the Aging (DFTA),2,2,0.0,1.0
2,Department of City Planning (DCP),2,1,0.0,0.5
3,Department of Consumer Affairs (DCA),1,0,0.0,0.0
4,Department of Design and Construction (DDC),3,0,3.0,0.0


In [25]:
#### DASHBOARD ####

#### Step 1. Get citywide metrics

# total number of rows
cw_numrows = quantity_agency_df['row_count'].sum()

# total number of datasets
cw_numdatasets = quantity_agency_df['numdatasets'].sum()
# percent updated on time
cw_freshness = freshness_dataset_df[freshness_dataset_df['fresh']=='Yes'].shape[0]/\
                    freshness_df.shape[0]
# percent released on time
cw_compliance = tracker_12mo_dataset_df['within_grace_period_num'].sum()/ \
                tracker_12mo_dataset_df.shape[0]

# number of assets that were supposed to be released but were not as of today 
cw_overdue = tracker_12mo_dataset_df[tracker_12mo_dataset_df['release_status']=='Scheduled for release'].shape[0]

citywide = pd.DataFrame([['Citywide',
                         cw_numrows,
                         cw_numdatasets,
                         cw_freshness,
                         cw_compliance,
                         cw_overdue]],
                       columns=['Scope',
                                'Number of published rows',
                                'Number of published datasets',
                                'Percent of datasets updated on time',
                                'Percent of datasets released on time in the last 12 months',
                                'Number of overdue datasets'])

citywide

,Scope,Number of published rows,Number of published datasets,Percent of datasets updated on time,Percent of datasets released on time in the last 12 months,Number of overdue datasets
0,Citywide,3.352433e+09,2520,0.426413,0.572816,31


In [26]:
#### Step 2. Build complete agency-level dataset

all_agency_df = quantity_agency_df.merge(freshness_agency_df, 
                                        on='datasetinformation_agency',
                                        how='outer')\
                                  .merge(tracker_12mo_agency_df, 
                                        on='datasetinformation_agency',
                                        how='outer')

all_agency_df['overdue_datasets'] = all_agency_df['overdue_datasets'].fillna(0)
all_agency_df['numdatasets'] = all_agency_df['numdatasets'].fillna(0)
all_agency_df['numrows'] = all_agency_df['row_count'].fillna(0)
all_agency_df['total_auto_count'] = all_agency_df['total_auto_count'].fillna(0)
all_agency_df['fresh_count'] = all_agency_df['fresh_count'].fillna(0)
all_agency_df['tracker_dataset_count'] = all_agency_df['tracker_dataset_count'].fillna(0)
all_agency_df['tracker_count_ontime'] = all_agency_df['tracker_count_ontime'].fillna(0)
all_agency_df['fresh_pct'] = all_agency_df['fresh_pct'].fillna('No automated datasets')
all_agency_df['pct_ontime'] = all_agency_df['pct_ontime'].fillna('No datasets in the tracker')
all_agency_df.shape

(93, 11)

In [27]:
all_agency_df.rename(columns={'datasetinformation_agency':'agency'},inplace=True)
all_agency_df = all_agency_df[[
                'agency',
                'numdatasets',
                'numrows',
                'total_auto_count',
                'fresh_count',
                'fresh_pct',
                'tracker_dataset_count',
                'tracker_count_ontime',
                'overdue_datasets',
                'pct_ontime'    
]]

In [28]:
#### Step 3. Build complete dataset-level dataset

# aggregate freshness data and tracker data (for released datasets only)
all_datasets_df = quantity_dataset_df.merge(freshness_dataset_df[['uid',
                                                                  'update_automation',
                                                                  'update_updatefrequency',
                                                                  'fresh']], 
                                        on='uid',
                                        how='outer')\
                                  .merge(tracker_12mo_dataset_df[['uid',
                                                                  'dataset_description',
                                                                  'latest_plan_date',   
                                                                  'release_status',
                                                                  'release_date',
                                                                  'within_grace_period',
                                                                  'within_grace_period_num']], 
                                        on='uid',
                                        how='left')
all_datasets_df.shape

(2520, 17)

In [29]:
tracker_12mo_dataset_df.head()

,dataset,dataset_description,latest_plan_date,release_status,release_date,within_grace_period,within_grace_period_num,url,uid,type,datasetinformation_agency
0,Street Improvement Project (SIP),All completed and planned street changes that ...,2019-12-31,Released,2018-05-31,Yes,True,https://data.cityofnewyork.us/Transportation/V...,wqhs-q6wd,map,Department of Transportation (DOT)
1,Highway Signs Installations,Database that tracks all signs installed in-ho...,2019-12-31,Released,2019-01-04,Yes,True,https://data.cityofnewyork.us/Transportation/H...,jkdk-6p97,dataset,Department of Transportation (DOT)
3,Street Name Signs (SNS) Management System,This application assists the Street Name Signs...,2019-12-31,Released,2019-02-27,Yes,True,https://data.cityofnewyork.us/Transportation/S...,guiy-frxt,dataset,Department of Transportation (DOT)
5,Waterborne Disease Risk Assessment Report,Tables from Waterborne Disease Risk Assessment...,2019-11-30,Released,2019-05-14,Yes,True,https://data.cityofnewyork.us/Health/DOHMH-and...,hphp-39kk,dataset,Department of Health and Mental Hygiene (DOHMH)
6,WalkNYC Signs,This data set contains the locations of existi...,2019-12-31,Released,2019-06-05,Yes,True,https://data.cityofnewyork.us/Transportation/W...,q49j-2bun,map,Department of Transportation (DOT)


In [30]:
# append non-released datasets data
# doing it as a separate step to keep more accurate data for released datasets
all_datasets_df = all_datasets_df.append(tracker_12mo_dataset_df[~tracker_12mo_dataset_df['uid'].isin(all_datasets_df['uid'])])\
                                 .reset_index(drop=True)

# merge name and dataset columns since they contain the same information
all_datasets_df.loc[all_datasets_df.name.isna(),'name'] = all_datasets_df['dataset']

# merge automation/update data for "historical" and "as needed" datasets
all_datasets_df = all_datasets_df.merge(public_df[['uid','update_automation','update_updatefrequency']], on='uid', how='left')

all_datasets_df['automation'] = np.where(all_datasets_df['update_automation_x'].isna(),
                                         all_datasets_df['update_automation_y'],
                                         all_datasets_df['update_automation_x'])
all_datasets_df['update_frequency'] = np.where(all_datasets_df['update_updatefrequency_x'].isna(),
                                         all_datasets_df['update_updatefrequency_y'],
                                         all_datasets_df['update_updatefrequency_x'])
# recode missing dates into string NA to properly read format in GDS
all_datasets_df['release_date_fix'] = pd.to_datetime(all_datasets_df['release_date'], errors='coerce')

all_datasets_df.shape

(2551, 23)

In [31]:
all_datasets_df.head()

,uid,datasetinformation_agency,name,url,type,update_datemadepublic,last_data_updated_date,row_count,update_automation_x,update_updatefrequency_x,...,release_status,release_date,within_grace_period,within_grace_period_num,dataset,update_automation_y,update_updatefrequency_y,automation,update_frequency,release_date_fix
0,2232-dj5q,Mayor's Office of Operations (OPS),Local Law 37 - DYCD Report,https://data.cityofnewyork.us/d/2232-dj5q,dataset,2018-12-04,2020-11-24,423.0,No,Monthly,...,NaN,NaN,NaN,NaN,NaN,No,Monthly,No,Monthly,NaT
1,7vu2-3skk,Office of Emergency Management (OEM),Hurricane Inundation Zones WNW,https://data.cityofnewyork.us/d/7vu2-3skk,map,2016-01-15,2016-01-15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,No,Historical Data,No,Historical Data,NaT
2,wugy-h4kt,Department of Education (DOE),2010-2011 School Closure Discharge Reporting C...,https://data.cityofnewyork.us/d/wugy-h4kt,dataset,2018-03-20,2018-03-20,NaN,No,Annually,...,NaN,NaN,NaN,NaN,NaN,No,Annually,No,Annually,NaT
3,exe6-f2vx,Department of Education (DOE),2014 - 2015 Arts Reports,https://data.cityofnewyork.us/d/exe6-f2vx,dataset,2019-04-16,2019-04-01,34.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,No,Historical Data,No,Historical Data,NaT
4,jkk2-3xzd,Department of Education (DOE),2015-2016 Discharge Reporting School Level - HS,https://data.cityofnewyork.us/d/jkk2-3xzd,dataset,2019-06-28,2019-06-28,12383.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,No,Historical Data,No,Historical Data,NaT


In [32]:
all_datasets_df['fresh'] = all_datasets_df['fresh'].fillna('No regular updates')
all_datasets_df['within_grace_period'] = all_datasets_df['within_grace_period'].fillna('Not in Open Plan Tracker')
all_datasets_df.shape

(2551, 23)

In [33]:
all_datasets_df.rename(columns={
 'datasetinformation_agency':'agency',
 'uid':'u_id',
 'url':'dataset_link',
 'update_datemadepublic':'date_made_public_dt',
 'row_count':'numrows',
 'last_data_updated_date':'last_update_date_data_dt',
 'latest_plan_date':'latest_plan_date_dt',
 'release_date_fix':'release_date_dt_fix'
},inplace=True)

all_datasets_df= all_datasets_df[[
    'agency',
    'u_id',
    'name',
    'dataset_description',
    'dataset_link',
    'type',
    'date_made_public_dt',
    'numrows',
    'automation',
    'update_frequency',
    'last_update_date_data_dt',
    'fresh',
    'latest_plan_date_dt',
    'release_status',
    'release_date_dt_fix',
    'within_grace_period'
]]

In [34]:
all_datasets_df.head()

,agency,u_id,name,dataset_description,dataset_link,type,date_made_public_dt,numrows,automation,update_frequency,last_update_date_data_dt,fresh,latest_plan_date_dt,release_status,release_date_dt_fix,within_grace_period
0,Mayor's Office of Operations (OPS),2232-dj5q,Local Law 37 - DYCD Report,NaN,https://data.cityofnewyork.us/d/2232-dj5q,dataset,2018-12-04,423.0,No,Monthly,2020-11-24,Yes,NaN,NaN,NaT,Not in Open Plan Tracker
1,Office of Emergency Management (OEM),7vu2-3skk,Hurricane Inundation Zones WNW,NaN,https://data.cityofnewyork.us/d/7vu2-3skk,map,2016-01-15,NaN,No,Historical Data,2016-01-15,No regular updates,NaN,NaN,NaT,Not in Open Plan Tracker
2,Department of Education (DOE),wugy-h4kt,2010-2011 School Closure Discharge Reporting C...,NaN,https://data.cityofnewyork.us/d/wugy-h4kt,dataset,2018-03-20,NaN,No,Annually,2018-03-20,No,NaN,NaN,NaT,Not in Open Plan Tracker
3,Department of Education (DOE),exe6-f2vx,2014 - 2015 Arts Reports,NaN,https://data.cityofnewyork.us/d/exe6-f2vx,dataset,2019-04-16,34.0,No,Historical Data,2019-04-01,No regular updates,NaN,NaN,NaT,Not in Open Plan Tracker
4,Department of Education (DOE),jkk2-3xzd,2015-2016 Discharge Reporting School Level - HS,NaN,https://data.cityofnewyork.us/d/jkk2-3xzd,dataset,2019-06-28,12383.0,No,Historical Data,2019-06-28,No regular updates,NaN,NaN,NaT,Not in Open Plan Tracker


In [35]:
#### Step 4. Upload data to Google Spreadsheets

# credentials.gs_upload(df=citywide, 
#                     wks_name='_citywide_')
# print('Upload complete for citywide dataset')

# credentials.gs_upload(df=all_agency_df, 
#                     wks_name='_agency_')
# print('Upload complete for agency dataset')

# credentials.gs_upload(df=all_datasets_df, 
#                     wks_name='_datasets_')
# print('Upload complete for datasets dataset')

# credentials.gs_upload(df=dates_df, 
#                     wks_name='_dates_')
# print('Upload complete for dates dataset')

In [36]:
print(f"Dashboard was updated on: {date.today()}")

Dashboard was updated on: 2020-11-25


In [37]:
citywide

,Scope,Number of published rows,Number of published datasets,Percent of datasets updated on time,Percent of datasets released on time in the last 12 months,Number of overdue datasets
0,Citywide,3.352433e+09,2520,0.426413,0.572816,31
